In [1]:
import gym
import numpy as np


In [10]:
class FrozenLakeModel:
    def __init__(self, transitions):
        self.model = transitions

    def step(self, obs, action):
        return self.model[obs][action][0]
    
    def actions(self, obs):
        return list(self.model[obs].keys())


class MCTS:
    def __init__(self, cur_state, model):
        self.cur_state = cur_state
        self.model = model
    
    def search(self):

        root = Node(self.cur_state)
        while self._running(root):
            next_node = self._treePolicy(root)
            reward = self._rolloutPolicy(next_node)
            self._backup(next_node, reward)
        
        return self._best_action(root), root

    def _running(self):
        pass

    def _best_action(self, root):
        pass

    def _treePolicy(self, cur_node):
        pass

    def _rolloutPolicy(self, cur_node):
        pass

    def _backup(self, cur_node, reward):
        pass

class Node:
    def __init__(self, state, action=None):
        self.state = state
        self.action = action


def run_episode(env, model):
    obs = env.reset()
    done = False

    ep_r = 0
    while not done:
        action, values = MCTS(env, model).search()
        obs, r, done, info = env.step(action)

        ep_r += r
    
    return ep_r, values # TODO: might want to return list of values for each step and stitch together
        

